In [3]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import numpy as np

from time import sleep
import csv
from datetime import datetime, timedelta
import os
import requests
from difflib import SequenceMatcher
import pandas_datareader.data as web
pd.set_option('display.max_rows', None)
!pip install html5lib
import html5lib

In [2]:
df = pd.read_csv('nasdaq.csv')
df.shape

(3035, 10)

df.dtypes

In [4]:
'''Checking for duplicate values'''
print("number of duplicated companies in the dataset:", df['Company Name'].duplicated().sum())


number of duplicated companies in the dataset: 4


In [5]:
df[df['Company Name'].duplicated(keep=False)]


,Unnamed: 0,Deal ID,Symbol,Company Name,Exchange/ Market,Price,Shares,Date,Offer Amount,Actions
87,11,832935-64691,AMBO,Ambow Education Holding Ltd.,New York Stock Exchange,10.00,10677207.0,2010-08-05,106772070.0,Priced
88,12,829325-64285,DMED,Steakholder Foods Ltd.,Nasdaq SmallCap Market,7.00,1500906.0,2010-08-05,10506342.0,Priced
213,19,696018-66111,SDT,SANDRIDGE ENERGY INC,New York Stock Exchange,21.00,15000000.0,2011-04-07,315000000.0,Priced
264,2,696018-67249,PER,SANDRIDGE ENERGY INC,New York Stock Exchange,18.00,30000000.0,2011-08-11,540000000.0,Priced
352,11,696018-68976,SDR,SANDRIDGE ENERGY INC,New York Stock Exchange,21.00,26000000.0,2012-04-18,546000000.0,Priced
1587,37,832935-84606,AMBO,Ambow Education Holding Ltd.,NYSE MKT,4.25,1800000.0,2018-06-01,7650000.0,Priced
2499,83,1147118-96122,MITC,Steakholder Foods Ltd.,NASDAQ Capital,10.30,2427185.0,2021-03-12,25000006.0,Priced


Commentary: Ambow Education LTD was enlisted twice. The first one it was liquedated after a scandal in 2014. Sandridge Energy Inc - all three are different trusts that belong to one company. Technically they are different

In [6]:
print("number of duplicate tickers in the dataset:", df['Symbol'].duplicated().sum())

number of duplicate tickers in the dataset: 44


In [7]:
df[df['Symbol'].duplicated(keep=False)].sort_values(by = ['Symbol'])

,Unnamed: 0,Deal ID,Symbol,Company Name,Exchange/ Market,Price,Shares,Date,Offer Amount,Actions
1791,3,818205-89797,ADPT,Adaptive Biotechnologies Corp,NASDAQ Global Select,20.00,15000000.0,2019-06-27,3.000000e+08,Priced
796,8,935353-75509,ADPT,Adeptus Health Inc.,New York Stock Exchange,22.00,4900000.0,2014-06-25,1.078000e+08,Priced
2765,28,1164345-98052,AHPAU,"OmniAb, Inc.",NASDAQ Capital,10.00,20000000.0,2021-08-10,2.000000e+08,Priced
1249,14,1002700-81618,AHPAU,Organogenesis Holdings Inc.,NASDAQ Capital,10.00,30000000.0,2016-10-11,3.000000e+08,Priced
87,11,832935-64691,AMBO,Ambow Education Holding Ltd.,New York Stock Exchange,10.00,10677207.0,2010-08-05,1.067721e+08,Priced
1587,37,832935-84606,AMBO,Ambow Education Holding Ltd.,NYSE MKT,4.25,1800000.0,2018-06-01,7.650000e+06,Priced
1700,2,1070273-88457,ANDAU,"STRYVE FOODS, INC.",NASDAQ Capital,10.00,10000000.0,2019-01-29,1.000000e+08,Priced
335,16,868381-68659,ANDAU,Tecnoglass Inc.,Nasdaq SmallCap Market,10.00,4000000.0,2012-03-19,4.000000e+07,Priced
1444,24,662564-84953,AQ,AQUANTIA CORP,NYSE,9.00,6818000.0,2017-11-03,6.136200e+07,Priced
342,0,876716-69363,AQ,Acquity Group Ltd,American Stock Exchange,6.00,5555556.0,2012-04-27,3.333334e+07,Priced


Commentary: There are companies that either got defunct and their tickers were given to new IPOs (like EVER) or there are companies that went to IPO twice (like AMBOW). Either way, because I will scrape stock data based on tickers from Yahoo Finance, which has information only for the most recent company, I keep here only those companies that went to IPO later. Others, I declare defunct.

In [8]:
defunct_oldticker = df['Symbol'][df['Symbol'].duplicated(keep='last')]


In [9]:
df = df[~df['Symbol'].duplicated(keep='last')]

Checking misalignment between company name and its ticker:
After scraping IPO companies I noticed that there are some misalignments between company names and their tickers. Tickers are different than company names, usually its an abbreviation. This could be due to the fact that some companies are now defunct, changed their names or spelling mistake etc.

So in this step, I check if letters in the symbol correspond to company name. If they don't, I construct a dict of suspect companies. In the next step, I check these suspicious tickers on Yahoo Finance, retrieve their company names their and compare to company names on Nasdaq list

In [10]:
df['Company Name'] = df['Company Name'].str.upper()

In [11]:
a = df[df['Symbol'].isna()]
a

,Unnamed: 0,Deal ID,Symbol,Company Name,Exchange/ Market,Price,Shares,Date,Offer Amount,Actions
3004,6,1219804-103318,NaN,NANO LABS LTD,NASDAQ Global,11.5,1770000.0,2022-07-12,20355000.0,Priced


In [12]:
df = df.dropna(axis=0)

In [13]:
#tmpdf = df.iloc[,:]
susp = {}
for i, symbol in enumerate(df.Symbol):
    #print(i,symbol)
    for letter in symbol:
        #print(letter)
        #print(df.iloc[i, 3])
        if letter not in df.iloc[i, 3]:
            susp.update({df.iloc[i, 3]: symbol})
        #print(susp)
#susp

In [14]:
print(f"{len(susp)} companies have suspicious tickers. Will be checked")

921 companies have suspicious tickers. Will be checked


In [ ]:
url_1 = 'https://finance.yahoo.com/quote/%s/'
url_2 = 'profile?p=%s'
unmatched = {}
unmatched_tickers = []

for i, ticker in enumerate(list(susp.values())):
    if i%10 == 0:
        print(f"scraping {ticker} - {i+1}/{len(susp)}")
    #print(ticker)
    res = requests.get(url_1 % ticker + url_2 % ticker, headers={'User-Agent': 'Custom'})
    #print(res)
    content = res.content
    #print(content)
    soup = BeautifulSoup(content)
    data = soup.find_all("h1", {"class" : "D(ib) Fz(18px)"})
    #print(data)
    if data:
        txt = str([x.get_text() for x in data]).replace(ticker, "").lower()
        regex = re.compile("\w+")
        txt = " ".join(regex.findall(txt)).replace("corporation", "corp").replace("limited", "ltd")
        name = df['Company Name'][df['Symbol'] == ticker].values[0].lower().replace(",", "").replace(".", "").replace("corporation", "corp").replace("limited", "ltd")
        #print(name)
        #checking with SequenceMatcher if two strings match for at least 80%
        if SequenceMatcher(None, txt, name).ratio() < 0.8:
            unmatched.update({name: txt})
            unmatched_tickers.append(ticker)


In [16]:
print(f'there are {len(unmatched_tickers)} unmatched tickers. Will be removed')
unmatched

there are 171 unmatched tickers. Will be removed


{'ifm investments ltd': '229323',
 'newegg commerce inc': '',
 'r1 rcm holdco inc': '778108',
 'westmoreland resource partners lp': '',
 'autonavi holdings ltd': '',
 'g-estate liquidation stores inc': 'goldman sachs manufacturing revolution etf',
 'trius therapeutics inc': '',
 'tal education group': '',
 'campus crest communities inc': '1120',
 'dunxin financial holdings ltd': '',
 'lentuo international inc': '',
 'global brokerage inc': '',
 'leaf group ltd': '874',
 'emergent capital inc': '27579',
 'mmodal inc': 'medx holdings inc',
 'wright medical group nv': 'taronis technologies inc',
 'qihoo 360 technology co ltd': '',
 'responsys inc': '',
 'box ships inc': '262597',
 'ocera therapeutics inc': '',
 'terravia holdings inc': '',
 'link motion inc': '',
 'csi compressco lp': '',
 'c&j old coinc': '',
 'vereit inc': 'american realty capital propert',
 'east dubuque nitrogen partners lp': '',
 'crestwood midstream partners lp': '',
 'capri holdings ltd': 'michael kors holdings ltd

In [17]:
df = df[~df['Symbol'].isin(unmatched_tickers)]


In [18]:
df.reset_index(drop=True, inplace=True)


In [19]:
tmpdf = df.iloc[[2469],:]
tmpdf

,Unnamed: 0,Deal ID,Symbol,Company Name,Exchange/ Market,Price,Shares,Date,Offer Amount,Actions
2469,0,1166804-98301,DOLE,DOLE PLC,NYSE,16.0,25000000.0,2021-07-30,400000000.0,Priced


In [ ]:
df_yahoo1 = pd.DataFrame()
notscraped1 = []
for i, ticker in enumerate(df.Symbol):
    global st
    #print(i,ticker)
    try:
        if i%10 == 0:
            print(f"scraping {ticker} - {i+1}/{len(df.Symbol)}")
        st = web.DataReader(ticker, "yahoo", start = '2010-01-01')
        #print(st)
        st['indx'] = range(len(st))
        #print(st['indx'])
        inweek_idx = st['indx'][str(df[df['Symbol'] == ticker]['Date'].values[0])] + 5
        #print(inweek_idx)
        inmonth_idx = st['indx'][str(df[df['Symbol'] == ticker]['Date'].values[0])] + 21
        #inyear_idx = st['indx'][str(df[df['Symbol'] == ticker]['Date'].values[0])] + 365
        #print(inyear_idx)
        firstday_open = st['Open'][str(df[df['Symbol'] == ticker]['Date'].values[0])]
        #print(firstday_open)
        firstday_adjclose = st['Adj Close'][str(df[df['Symbol'] == ticker]['Date'].values[0])]
        #print(firstday_adjclose)
        inweek_open = st['Open'][st['indx'] == inweek_idx][0]
        inweek_adjclose = st['Adj Close'][st['indx'] == inweek_idx][0]
        #print(inweek_open)

        inmonth_open = st['Open'][st['indx'] == inmonth_idx][0]
        inmonth_adjclose = st['Adj Close'][st['indx'] == inmonth_idx][0]
        
        #inyear_open = st['Open'][st['indx'] == inyear_idx][0]
        #inyear_adjclose = st['Adj Close'][st['indx'] == inyear_idx][0]
        #print(inyear_adjclose)
        
        ticker_df1 = pd.DataFrame({ticker: {'firstday_adjclose': firstday_adjclose, 'firstday_open': firstday_open, 
                                           'inweek_adjclose': inweek_adjclose, 'inweek_open': inweek_open, 
                                           'inmonth_adjclose': inmonth_adjclose, 'inmonth_open': inmonth_open}}).T

        df_yahoo1 = df_yahoo1.append(ticker_df1)
        #print(df_yahoo1)
            
    except:
        notscraped1.append(ticker)
        continue

In [24]:
df_yahoo1.tail()

,firstday_adjclose,firstday_open,inmonth_adjclose,inmonth_open,inweek_adjclose,inweek_open
YOSH,4.17,5.200000,1.178,1.200000,1.780000,2.010000
BIAF,8.30,8.400000,2.510,2.900000,4.780000,5.500000
MBLY,28.25,28.250000,29.400,29.500000,25.490000,25.840000
PRME,15.37,18.969999,19.400,20.059999,17.299999,16.450001
CTM,1.20,1.360000,0.832,0.808000,0.861000,0.940000


In [25]:
print(f"From {len(df.Symbol)} IPO companies listed on Nasdaq, {len(notscraped1)} companies were not scraped on Yahoo Finance")

From 2819 IPO companies listed on Nasdaq, 1053 companies were not scraped on Yahoo Finance


In [26]:
'''Merging both data'''
df.index = df.Symbol
df.index

Index(['CELM', 'AMCF', 'CHC', 'CLU', 'SYA', 'GNRC', 'GRM', 'QNST', 'TRNO',
       'IRWD',
       ...
       'NXL', 'CRBG', 'THRD', 'LNKB', 'WLDS', 'YOSH', 'BIAF', 'MBLY', 'PRME',
       'CTM'],
      dtype='object', name='Symbol', length=2819)

In [127]:
df_ipo = df.join(df_yahoo1, how='inner')


In [ ]:
df_ipo.iloc[1209,:]

1.2. Scraping profile and industry from Yahoo Finance \n
In this step, I scrape sector, industry, employees as of now, if available CEO pay and CEO year born from Yahoo Finance.

In [ ]:

df_ipo = df_ipo.drop(labels = 'DFH', axis=0)

df_ipo

In [ ]:
follow = []
sector = []
industry = []
employees2019 = []
CEO_pay = []
CEO_born = []

url_1 = 'https://finance.yahoo.com/quote/%s/'
url_2 = 'profile?p=%s'

for i, ticker in enumerate(df_ipo.Symbol):
    print(f"scraping {ticker} - {i+1}/{len(df_ipo.Symbol)}")
    result = requests.get(url_1 % ticker + url_2 % ticker, headers={'User-Agent': 'Custom'})
    content = result.content
    #print(content)

    soup = BeautifulSoup(content)
    data = soup.find_all('span', {"class":"Fw(600)"})
    #print(data)
    
    if data:
        txt = [x.get_text() for x in data]
        #print(txt)
        if len(txt) == 4:
            follow.append(txt[0])
            sector.append(txt[1])
            #print(sector)
            industry.append(txt[2])
            employees2019.append(txt[3])
        else:
            sector.append(np.nan)
            industry.append(np.nan)
            employees2019.append(np.nan)
        

    else:
        sector.append(np.nan)
        industry.append(np.nan)
        employees2019.append(np.nan)
    
    try:
        if len(txt) == 4:
            table = pd.read_html(content)[0]
            #print(table)
            #checking if table has CEO 
            if ('Title' in table) and (table['Title'].str.contains('CEO').sum() == 1):
                CEO_pay.append(table['Pay'][table['Title'].str.contains('CEO')].values[0]) 
                CEO_born.append(table['Year Born'][table['Title'].str.contains('CEO')].values[0])
            else:
                CEO_pay.append(np.nan) 
                CEO_born.append(np.nan)
            
    except ValueError:
        CEO_pay.append(np.nan) 
        CEO_born.append(np.nan)
    
print(f"checking lengths sector: {len(sector)}, industry: {len(industry)}, employees2019: {len(employees2019)}, CEO pay: {len(CEO_pay)}, CEO_born: {len(CEO_born)}")


In [138]:
employees2019_clean = []
for emp in employees2019:
    if emp is not np.nan:
        if len(emp) > 0:
            emp = int(emp.replace(',', ''))
        else:
            emp = np.nan
    employees2019_clean.append(emp)
    


In [189]:
len(CEO_pay_clean)

1696

In [ ]:
CEO_pay_clean1 = []
for i, cp in enumerate(CEO_pay1):
    if cp == cp:
        if isinstance(cp, float):
            print(i, cp)
            cp_clean = cp*1000
        elif 'M' in cp:
            print(i, cp)
            cp_clean = float(cp.replace("M", ""))*1000000
        elif 'k' in cp:
            print(i, cp)
            cp_clean = float(cp.replace("k", ""))*1000        
        CEO_pay_clean1.append(cp_clean)
    elif cp != cp:
        print(i, 'this one', cp)
        CEO_pay_clean1.append(cp)
        


In [151]:
df_ipo['sector'] = sector
df_ipo['industry'] = industry
df_ipo['employees2019'] = employees2019_clean


In [198]:
df_ipo['CEO_pay'] = CEO_pay_clean1
df_ipo['CEO_born'] = CEO_born1

In [200]:
df_ipo.head()

,Unnamed: 0,Deal ID,Symbol,Company Name,Exchange/ Market,Price,Shares,Date,Offer Amount,Actions,...,firstday_open,inmonth_adjclose,inmonth_open,inweek_adjclose,inweek_open,sector,industry,employees2019,CEO_pay,CEO_born
AMCF,2,810870-61911,AMCF,ANDATEE CHINA MARINE FUEL SERVICES CORP,Nasdaq National Market,6.30,3134921.0,2010-01-26,19750002.0,Priced,...,6.50000,5.820000,5.840000,6.000000,6.100000,Energy,Oil & Gas Equipment & Services,189.0,NaN,NaN
GNRC,2,814154-62379,GNRC,GENERAC HOLDINGS INC.,New York Stock Exchange,13.00,18750000.0,2010-02-11,243750000.0,Priced,...,13.00000,9.877322,14.000000,8.717610,13.000000,Industrials,Specialty Industrial Machinery,8955.0,2520000.0,1972.0
QNST,4,158807-62653,QNST,"QUINSTREET, INC",Nasdaq National Market,15.00,10000000.0,2010-02-11,150000000.0,Priced,...,15.00000,14.960000,15.290000,13.900000,14.500000,Communication Services,Advertising Agencies,791.0,655500.0,1960.0
TRNO,5,815376-62539,TRNO,TERRENO REALTY CORP,New York Stock Exchange,20.00,8750000.0,2010-02-10,175000000.0,Priced,...,18.75000,14.457020,19.450001,13.957734,18.750000,Real Estate,REIT—Industrial,34.0,808700.0,1961.0
IRWD,7,788969-62670,IRWD,IRONWOOD PHARMACEUTICALS INC,Nasdaq National Market,11.25,16666667.0,2010-02-03,187500004.0,Priced,...,10.38526,11.005025,11.323283,9.798995,9.757119,Healthcare,Drug Manufacturers—Specialty & Generic,219.0,1570000.0,1957.0


In [201]:
df_ipo.to_csv('data_ipo.csv', index=False)


In [180]:
tmpf = df_ipo.iloc[[15],:]
tmpf

,Unnamed: 0,Deal ID,Symbol,Company Name,Exchange/ Market,Price,Shares,Date,Offer Amount,Actions,firstday_adjclose,firstday_open,inmonth_adjclose,inmonth_open,inweek_adjclose,inweek_open,sector,industry,employees2019
MITL,6,106107-62946,MITL,MITEL NETWORKS CORP,Nasdaq National Market,14.0,10526316.0,2010-04-22,147368424.0,Priced,12.3,13.08,10.99,11.02,12.23,11.9,NaN,NaN,NaN


In [ ]:
CEO_pay1 = []
CEO_born1 = []

url_1 = 'https://finance.yahoo.com/quote/%s/'
url_2 = 'profile?p=%s'

for i, ticker in enumerate(df_ipo.Symbol):
    print(f"scraping {ticker} - {i+1}/{len(df_ipo.Symbol)}")
    result1 = requests.get(url_1 % ticker + url_2 % ticker, headers={'User-Agent': 'Custom'})
    content1 = result1.content
    #print(content1)
    print(result1.status_code)

    soup = BeautifulSoup(content1)
    #print(soup)
    
    try:
        table1 = pd.read_html(content1)[0]
        #print(table1)
        #checking if table has CEO 
        if ('Title' in table1) and (table1['Title'].str.contains('CEO').sum() == 1):
            CEO_pay1.append(table1['Pay'][table1['Title'].str.contains('CEO')].values[0]) 
            CEO_born1.append(table1['Year Born'][table1['Title'].str.contains('CEO')].values[0])
        else:
            CEO_pay1.append(np.nan) 
            CEO_born1.append(np.nan)
            
    except:
        CEO_pay1.append(np.nan) 
        CEO_born1.append(np.nan)

In [4]:
df_t = pd.read_csv('data_ipo.csv')

In [159]:
tmpf = df_t.iloc[[10],:]
tmpf

,Unnamed: 0,Deal ID,Symbol,Company Name,Exchange/ Market,Price,Shares,Date,Offer Amount,Actions,...,inmonth_adjclose,inmonth_open,inweek_adjclose,inweek_open,sector,industry,employees2019,CEO_pay,CEO_born,SP_Week_Chg
10,7,632487-62514,MXL,MAXLINEAR INC,New York Stock Exchange,14.0,6444100.0,2010-03-24,90217400.0,Priced,...,17.1,17.049999,17.780001,18.0,Technology,Semiconductors,1503.0,638790.0,1969.0,2.102632


In [173]:
add = []

url_1 = 'https://finance.yahoo.com/quote/%s/'
url_2 = 'profile?p=%s'

for i, ticker in enumerate(tmpf.Symbol):
    print(f"scraping {ticker} - {i+1}/{len(tmpf.Symbol)}")
    result1 = requests.get(url_1 % ticker + url_2 % ticker, headers={'User-Agent': 'Custom'})
    content1 = result1.content
    #print(content1)
    #print(result1.status_code)

    soup = BeautifulSoup(content1)
    #print(soup)
    
    data = soup.find_all('p', {"D(ib) W(47.727%) Pend(40px)"})
    #print(data)
    
    if data:
        txt = [x.get_text(separator = '\n') for x in data]
        print(txt)
        add.append(txt)
    else:
        add.append(np.nan)

scraping MXL - 1/1
['5966 La Place Court\nSuite 100\nCarlsbad, CA 92008\nUnited States\n760 692 0711\nhttps://www.maxlinear.com']


In [ ]:
address = []

url_1 = 'https://finance.yahoo.com/quote/%s/'
url_2 = 'profile?p=%s'

for i, ticker in enumerate(df_t.Symbol):
    print(f"scraping {ticker} - {i+1}/{len(df_t.Symbol)}")
    result1 = requests.get(url_1 % ticker + url_2 % ticker, headers={'User-Agent': 'Custom'})
    content1 = result1.content
    #print(content1)
    #print(result1.status_code)

    soup = BeautifulSoup(content1)
    #print(soup)
    
    data = soup.find_all('p', {"class":"D(ib) W(47.727%) Pend(40px)"})
    #print(data)
    
    if data:
        txt = [x.get_text(separator = '\n') for x in data]
        address.append(txt[0])
    else:
        address.append(np.nan)
        
    
    

scraping AMCF - 1/1765
scraping GNRC - 2/1765
scraping QNST - 3/1765
scraping TRNO - 4/1765
scraping IRWD - 5/1765
scraping SSNC - 6/1765
scraping STNG - 7/1765
scraping HTHT - 8/1765
scraping FIBK - 9/1765
scraping CALX - 10/1765
scraping MXL - 11/1765
scraping AVEO - 12/1765
scraping ST - 13/1765
scraping ANTH - 14/1765
scraping AOSL - 15/1765
scraping MITL - 16/1765
scraping CDXS - 17/1765
scraping SPSC - 18/1765
scraping PRI - 19/1765
scraping JKS - 20/1765
scraping EXPR - 21/1765
scraping RRTS - 22/1765
scraping PLOW - 23/1765
scraping TSLA - 24/1765
scraping FN - 25/1765
scraping HPP - 26/1765
scraping CBOE - 27/1765
scraping BNO - 28/1765
scraping ENV - 29/1765
scraping GDOT - 30/1765
scraping AMRC - 31/1765
scraping WSR - 32/1765
scraping ELMD - 33/1765
scraping MMYT - 34/1765
scraping NXPI - 35/1765
scraping AMRS - 36/1765
scraping PACB - 37/1765
scraping BBRG - 38/1765
scraping VRA - 39/1765
scraping EFC - 40/1765
scraping DQ - 41/1765
scraping CCIH - 42/1765
scraping KEYW - 

In [62]:
pd.set_option('display.max_colwidth', None)
address = pd.DataFrame(address)
address

,0
0,No.68 of West Binhai RoadUnit C Xigang DistrictDalianChina86 11 8240 8939https://www.andatee.com
1,"S45 W29290 Highway 59Waukesha, WI 53189United States262 544 4811https://www.generac.com"
2,"950 Tower Lane6th FloorFoster City, CA 94404United States650 587 7700https://www.quinstreet.com"
3,"10500 NE 8th StreetSuite 301Bellevue, WA 98004United Stateshttps://www.terreno.com"
4,"100 Summer StreetSuite 2300Boston, MA 02110United States617 621 7722https://www.ironwoodpharma.com"
5,"80 Lamberton RoadWindsor, CT 06095United States860 298 4500https://www.ssctech.com"
6,"Le Millenium9, Boulevard Charles IIIMonaco 98000Monaco377 9798 5716https://www.scorpiotankers.com"
7,No. 699 Wuzhong RoadMinhang DistrictShanghai 201103China86 21 6195 2011https://ir.hworld.com
8,"401 North 31st StreetBillings, MT 59116-0918United States406 255 5311https://www.fibk.com"
9,"2777 Orchard ParkwaySan Jose, CA 95134United States408 514 3000https://www.calix.com"


In [179]:
tmp_ad = address.iloc[[1,2],:]
tmp_ad

AttributeError: 'dict' object has no attribute 'iloc'

In [ ]:
sp = pd.read_csv('spy.csv')
sp.head()

In [124]:
sp2 = sp[sp.columns[~sp.columns.isin(['Date'])]]
sp2.columns

Index(['Unnamed: 0', 'GSPC.Open', 'GSPC.High', 'GSPC.Low', 'GSPC.Close',
       'GSPC.Volume', 'GSPC.Adjusted'],
      dtype='object')

In [125]:
sp2.columns = sp2.columns.str.replace('Unnamed: 0', 'Date')

In [ ]:
sp = sp2
sp

In [127]:
sp.sort_values('Date', inplace=True)
sp.reset_index(drop=True, inplace=True)
sp.Date = pd.to_datetime(sp.Date)

In [128]:
df_t.Date = pd.to_datetime(df_t.Date)

In [129]:
def get_week_chg(ipo_dt):
    try: 
        day_ago_idx = sp[sp['Date'] == str(ipo_dt.date())].index[0] - 1
        #print(day_ago_idx)
        week_ago_idx = sp[sp['Date'] == str(ipo_dt.date())].index[0] - 8
        chg = (sp.iloc[day_ago_idx]['GSPC.Close'] - sp.iloc[week_ago_idx]['GSPC.Close'])/(sp.iloc[week_ago_idx]['GSPC.Close'])
        #print(chg)
        return chg * 100
    except:
        print('error', ipo_dt.date())     

In [132]:
df_t['SP_Week_Chg'] = df_t['Date'].map(get_week_chg)

In [133]:
df_t.head()

,Unnamed: 0,Deal ID,Symbol,Company Name,Exchange/ Market,Price,Shares,Date,Offer Amount,Actions,...,inmonth_adjclose,inmonth_open,inweek_adjclose,inweek_open,sector,industry,employees2019,CEO_pay,CEO_born,SP_Week_Chg
0,2,810870-61911,AMCF,ANDATEE CHINA MARINE FUEL SERVICES CORP,Nasdaq National Market,6.30,3134921.0,2010-01-26,19750002.0,Priced,...,5.820000,5.840000,6.000000,6.100000,Energy,Oil & Gas Equipment & Services,189.0,NaN,NaN,-4.268210
1,2,814154-62379,GNRC,GENERAC HOLDINGS INC.,New York Stock Exchange,13.00,18750000.0,2010-02-11,243750000.0,Priced,...,9.877322,14.000000,8.717610,13.000000,Industrials,Specialty Industrial Machinery,8955.0,2520000.0,1972.0,-1.933541
2,4,158807-62653,QNST,"QUINSTREET, INC",Nasdaq National Market,15.00,10000000.0,2010-02-11,150000000.0,Priced,...,14.960000,15.290000,13.900000,14.500000,Communication Services,Advertising Agencies,791.0,655500.0,1960.0,-1.933541
3,5,815376-62539,TRNO,TERRENO REALTY CORP,New York Stock Exchange,20.00,8750000.0,2010-02-10,175000000.0,Priced,...,14.457020,19.450001,13.957734,18.750000,Real Estate,REIT—Industrial,34.0,808700.0,1961.0,-0.311953
4,7,788969-62670,IRWD,IRONWOOD PHARMACEUTICALS INC,Nasdaq National Market,11.25,16666667.0,2010-02-03,187500004.0,Priced,...,11.005025,11.323283,9.798995,9.757119,Healthcare,Drug Manufacturers—Specialty & Generic,219.0,1570000.0,1957.0,1.058835


In [134]:
df_t.to_csv('ipo_data.csv', index=False)

In [177]:
address.to_csv('add.csv', index=False)

AttributeError: 'dict' object has no attribute 'to_csv'